# Setting up EC2 instances

In [1]:
import boto3
import os
import time
from fabric import Connection
from botocore.exceptions import ClientError
ec2 = boto3.client('ec2')

### Functionalities

In [2]:
instance_ami = 'ami-061eb2b23f9f8839c'
num_instance = 1
tier = 't2.micro'
key_pair = 'prodkey19'
security_group_name = 'sum1'
security_desc = 'Security group for Web Server'


In [3]:
# AWS EC2 list instance images
def list_images():
    res = ec2.describe_images(Owners=['self'])
    for img in res['Images']:
        print("Name: ",img['Name'])
        print("Image: ", img['ImageId'])
        print("Description: ", img['Description'])
        print("----")

In [4]:
# AWS EC2 save instance image
def save_instance(ins, name, desc='My new instance'):
    res = ec2.create_image(InstanceId=ins, Name=name, Description=desc)
    print("Created image: ",res['ImageId'])
    print("Waiting for it to be available...")

    # wait for it to be available
    available = 0
    while (not available):
        status = ec2.describe_images(ImageIds=[res['ImageId']])
        img = status['Images'][0]
        available = (img['State'] == 'available')
        time.sleep(1)

In [5]:
# AWS EC2 create key-pair
def create_key_pair(name):
    response = ec2.create_key_pair(
        KeyName=name
    )
    key = response['KeyMaterial']
    f= open(name+".pem","w")
    f.write(key)
    f.close
    os.system('chmod 400 %s.pem' %name)
    print('Key Pair %s Created' %name)
    return response

In [6]:
# AWS EC2 delete key-pair
def delete_key_pair(name):
    ec2.delete_key_pair(KeyName=name)
    os.remove(name+'.pem')
    print('Key Pair %s Deleted' %name)

In [7]:
# AWS EC2 create security group
def create_security_group(name, description, ip_permissions):
    response = ec2.describe_vpcs()
    vpc_id = response.get('Vpcs', [{}])[0].get('VpcId', '')

    response = ec2.create_security_group(GroupName=name,
                                         Description=description,
                                         VpcId=vpc_id)
    security_group_id = response['GroupId']
    print('Security Group Created %s in vpc %s.' % (security_group_id, vpc_id))

    data = ec2.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=ip_permissions)
    print('Ingress Successfully Set %s' %data)
    return security_group_id

In [8]:
def remove_security_group_permissions(security_group_id, ip_permissions):
    data = ec2.revoke_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=ip_permissions)
    print('Ingress Successfully removed %s' %data)
    return security_group_id

In [9]:
def set_security_group_permissions(security_group_id, ip_permissions):
    data = ec2.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=ip_permissions)
    print('Ingress Successfully Set %s' %data)
    return security_group_id

In [10]:
# AWS EC2 delete security group
def delete_security_group(security_group_id):
    response = ec2.delete_security_group(GroupId=security_group_id)
    print('Security Group %s Deleted' %security_group_id)

In [11]:
# AWS EC2 create instances
def create_instances(image, maxCount, instanceType, key, groupID):
    instances = ec2.run_instances(
        ImageId=image,
        MinCount=1,
        MaxCount=maxCount,
        InstanceType=instanceType,
        KeyName=key,
        SecurityGroupIds=[groupID]
    )
    instance_list = []
    for i in instances['Instances']:
        instance_list.append(i['InstanceId'])
    
    print('Instances Created %s' %instance_list)
    return instance_list

In [12]:
# AWS EC2 list instances
def list_ec2_instances():
    instances = {}
    res = ec2.describe_instances()
    for r in res['Reservations']:
        for ins in r['Instances']:
            if ins['State']['Name'] == 'running' or ins['State']['Name'] == 'pending':
                instances[ins['InstanceId']] = ins['PublicIpAddress']
    print('List of active Instances %s' %instances)
    return instances

In [13]:
# AWS EC2 terminate instances
def terminate_instances(instances):
    response = ec2.terminate_instances(
        InstanceIds=instances
    )
    print('Instances Deleted %s' %instances)
    return response

### Create Key Pair

In [14]:
# generate the new key pair
# create_key_pair(key_pair)

# Set up an Ubuntu node EC2 instance

### First set up security group

In [15]:
# set up permissions for the new security group for the instance/node
node_ip_permissions = [{'IpProtocol': 'tcp',
                   'FromPort': 22,
                   'ToPort': 22,
                   'IpRanges': [{
                       'CidrIp': '0.0.0.0/0',
                       'Description': 'SSH'}]},
                  {'IpProtocol': 'tcp',
                   'FromPort': 80,
                   'ToPort': 80,
                   'IpRanges': [{
                       'CidrIp': '0.0.0.0/0',
                       'Description': 'HTTP'}]},
                  {'IpProtocol': 'tcp',
                   'FromPort': 3000,
                   'ToPort': 3000,
                   'IpRanges': [{
                       'CidrIp': '0.0.0.0/0',
                       'Description': 'MySQL'}]}]



In [16]:
# create the security group using the previously made permissions
node_groupID = create_security_group(security_group_name, security_desc, node_ip_permissions)
node_groupID

Security Group Created sg-0fca978ad1156468b in vpc vpc-1d78777a.
Ingress Successfully Set {'ResponseMetadata': {'RequestId': '352d9f6b-99b8-4d6d-b6c5-22cb5db8e707', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'content-length': '259', 'date': 'Mon, 23 Dec 2019 11:26:46 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


'sg-0fca978ad1156468b'

Now to create the EC2 instance, in this case we try with 1 first by setting max count to 1

In [17]:
# create a new single instance
# we are using the ubuntu AMI for this and using t2.medium
instance_node_list = create_instances(instance_ami, num_instance, tier, key_pair, node_groupID)
instance_node = instance_node_list[0]
instance_node

Instances Created ['i-005d43e809101b8b3']


'i-005d43e809101b8b3'

In [18]:
# get the public IP of the instance
time.sleep(5)
instance_dic = list_ec2_instances()
node_ip = instance_dic[instance_node]
node_ip

List of active Instances {'i-064012ea2e13e959a': '18.141.90.224', 'i-0c028a47091f91dff': '54.169.70.231', 'i-005d43e809101b8b3': '52.77.251.91'}


'52.77.251.91'

# Using Fabric Connection function to run commands on the EC2 instance

In [19]:
c = Connection(
    host=node_ip,
    user="ubuntu",
    connect_kwargs={
        "key_filename": key_pair + ".pem",
    },
)


# Setting up MySQL Kindle Reviews

# Setting up Web Server 

In [20]:
# After creating EC2 instance

In [21]:
#  Creating
print('Setting up EC2 instance, please wait...')
print('Approx. waiting time: 2-5 mins')
time.sleep(65)
c.run('mkdir dbds_project')
c.run('cd dbds_project && git clone https://github.com/hello2508/Database.git')

Setting up EC2 instance, please wait...
Approx. waiting time: 2-5 mins


BlockingIOError: [Errno 11] Resource temporarily unavailable

In [ ]:
# Downloading the necessary packages
print('Updating...')
c.sudo('apt-get update')
c.sudo('apt-get install python3')

In [ ]:
# Install prerequisites before installing pip3
c.sudo('apt-get -y install build-essential libssl-dev libffi-dev python-dev')
c.sudo('apt-get update')
c.sudo('apt -y install python3-pip')

In [ ]:
# Create a virtual environment through Python3-venv
print('Creating virtual environment(webenv) for web server...')
c.sudo('apt install -y python3-venv')
c.run('cd dbds_project/Database/flaskproject && python3 -m venv webvenv')
time.sleep(5)
c.run('cd dbds_project/Database/flaskproject && source ./webvenv/bin/activate')

In [ ]:
# Get dependencies
print('Installing relevant packages...')
c.sudo('pip3 install --upgrade setuptools')
c.sudo('apt-get -y install python3-dev libmysqlclient-dev')
c.sudo('apt -y install libmpc-dev')
c.run('pip3 install --user gmpy2==2.1.0a2')
c.run('cd dbds_project/Database && pip3 install -r requirements.txt')
c.sudo('pip3 install pymongo')
c.sudo('apt-get install python3-mysql.connector')


In [ ]:
# Running Flask (ensure this is running in the correct directory)
time.sleep(3)
print('Setting up web server...')
print('Once completed, pls go to localhost:8888 to see the Web server.')
c.run('cd dbds_project/Database/flaskproject && export FLASK_APP=app.py')
c.run('cd dbds_project/Database/flaskproject && export FLASK_ENV=development')
# c.run('cd dbds_project/Database/flaskproject && flask run')
c.run('cd dbds_project/Database/flaskproject && python3 app.py')